In [1]:
import numpy as np

from os import listdir
from os.path import isfile, join
import networkx as nx

import dgl
from dgl.nn import RelGraphConv

import torch
import torch.nn as nn
import torch.nn.functional as F

import numpy as np


from dgl.nn import GraphConv, AvgPooling
from dgl.nn.pytorch import Sequential

from dgl.dataloading import GraphDataLoader
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
from sklearn.model_selection import train_test_split

from sklearn.model_selection import KFold, StratifiedKFold

import matplotlib.pyplot as plt

from torch.optim import lr_scheduler

import pickle

from copy import deepcopy

In [2]:
pickle_name = 'nx_url_dataset.pickle'
with open(pickle_name, 'rb') as f:
    train_set, test_set = pickle.load(f)

In [3]:
e2id = {'title':0, 'link':0, 'domain':0}

def add_encoding(dataset):
    for j in range(len(dataset)):
        g = dataset[j][0]
        for e in g.edges:
#             print(g.edges[e[0], e[1]]['link_type'])
            if g.edges[e[0], e[1]]['link_type'] == 'title':
                g.edges[e[0], e[1]]['type'] = e2id['title']
                
            else:
                g.edges[e[0], e[1]]['type'] = e2id['link']
                
        node_domains = dict()
        for i in range(len(g.nodes)):
            g.nodes[i]['h'] = torch.tensor([0., 0., 0.])
            g.nodes[i]['h'][g.nodes[i]['level']] = 1.
            if g.nodes[i]['domain'] not in node_domains:
                node_domains[g.nodes[i]['domain']] = [i]
            else:
                node_domains[g.nodes[i]['domain']].append(i)
            
        print(node_domains)
        for domain in node_domains:
            if len(node_domains[domain]) >1:
                for d1 in node_domains[domain]:
                    for d2 in node_domains[domain]:
                        if d1 == d2:
                            continue
                        g.add_edge(d1, d2)
                        g.edges[d1, d2]['type'] = e2id['domain']
                        
    return dataset
train_set = add_encoding(train_set)
test_set = add_encoding(test_set)

{'euronews.com': [0, 39], 'dw.com': [1], 'nytimes.com': [2, 24], 'businessinsider.com': [3], 'theguardian.com': [4, 25], 'dailymail.co.uk': [5, 17], 'sltrib.com': [6], 'salon.com': [7], 'wbur.org': [8], 'foreignpolicy.com': [9], 'livemint.com': [10], 'toledoblade.com': [11], 'jta.org': [12], 'nzhistory.govt.nz': [13], 'monocle.com': [14], 'independent.co.uk': [15], 'csmonitor.com': [16], 'the-sun.com': [18], 'gbnews.uk': [19], 'sputniknews.com': [20], 'thesun.co.uk': [21], 'cnbc.com': [22], 'edition.cnn.com': [23], 'netflix.com': [26], 'fmovies2.cx': [27], 'hrw.org': [28], 'careers.firstwatch.com': [29], 'wealthygorilla.com': [30], 'beforeitsnews.com': [31], 'en.reshuege.ru': [32], 'variety.com': [33], 'reuters.com': [34], 'logos.fandom.com': [35], 'euronews.al': [36], 'euractiv.com': [37], 'forums.verizon.com': [38], 'hu.wikipedia.org': [40], 'flickr.com': [41], 'parsatv.com': [42], 'broadbandtvnews.com': [43], 'ru.euronews.com': [44], 'baidu.com': [45, 46, 47]}
{'apnews.com': [0, 1],

{'vice.com': [0], 'cyberlances.wordpress.com': [1, 112, 128, 136, 137, 139], 'mikesmaxims.com': [2, 170, 171, 207], 'aeternusmalus.wordpress.com': [3, 180, 223], 'nypost.com': [4, 298, 299, 551], 'westernjournal.com': [5, 424, 532], 'popularmechanics.com': [6, 245, 497, 498, 499, 500], 'gizmodo.com.au': [7], 'washingtonpost.com': [8], 'breitbart.com': [9, 531], 'newser.com': [10], 'waguns.org': [11], 'twitter.com': [12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 224, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 392, 393, 394, 395, 396, 397

{'eureporter.co': [0], 'news-europe.fr': [1], 'reddit.com': [2]}
{'eureporter.co': [0], 'news-europe.fr': [1], 'cybernistas.com': [2], 'twitter.com': [3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51]}
{'arabic.rt.com': [0], 'eg24.news': [1, 2, 3], 'nytimes.com': [4, 7], 'cnn.com': [5], 'brookings.edu': [6, 12], 'gzeromedia.com': [8], 'r1news.com.br': [9], 'apnews.com': [10], 'strategic-culture.org': [11], 'meduza.io': [13], 'theguardian.com': [14], 'mydailyspins.com': [15], 'bluestacks.com': [16], 'rbth.com': [17], 'guiasteam.com': [18], 'naturalnews.com': [19], 'teacherluke.co.uk': [20], 'top-mmogames.ru': [21], 'history.com': [22], 'gameplay.tips': [23], 'dtf.ru': [24], 'byrut.org': [25], 'mrguider.org': [26], 'dota2.jonny.ru': [27], 'neomods.ru': [28], 'yesasia.ru': [29], 'gnw.su': [30], 'ixbt.com': [31], 'minecraft-statistic.net': [32], 'arteni

{'basicthinking.de': [0], 'drone-zone.de': [1, 10], 'drohnen-camp.de': [2], 'wiwo.de': [3], 'preisvergleich.ch': [4], 'airandmore.at': [5, 61], 'conrad.at': [6], 'spektrum.de': [7], 'mdpi.com': [8], 'lite-magazin.de': [9], 'faz.net': [11, 67], 'einbuergerungstest-online.eu': [12], 'nashaucheba.ru': [13], 'ppt-online.org': [14], 'colon.de': [15], 'slowgerman.com': [16], 'mydocx.ru': [17], 'kita.de': [18], 'planradar.com': [19], 'de.reshuege.ru': [20], 'vk.com': [21], 'reitschuster.de': [22], 'mondkalender.shop': [23], 'buhl.de': [24], 'cyberleninka.ru': [25], 'de.reshuoge.ru': [26], 'backwinkel.de': [27], 'docplayer.org': [28], 'class.rambler.ru': [29], 'watson.ch': [30], 'olimpiadnye-zadanija.ru': [31], 'deutsch-lernen-online.net': [32], 'infourok.ru': [33], 'deutschegrammatik20.de': [34], 'de-ege.sdamgia.ru': [35], 'iknigi.net': [36], 'roberthalf.de': [37], 'nsportal.ru': [38], 'web-globus.de': [39], 'zachteno.net': [40], 'deutsch-mit-marija.de': [41], 'drohnenversicherung.blog': [42]

{'thebureauinvestigates.com': [0, 151], 'lesoir.be': [1, 13, 207, 208, 209], 'redbioetica.com.ar': [2, 258], 'uvrm.wordpress.com': [3, 331, 332, 333, 334], 'odbornakomisia.sk': [4, 413, 432], 'nultatacka.rs': [5, 461, 462, 491, 511, 519], 'herowndestiny.com': [6, 10, 535, 536, 581, 862, 863, 864, 865, 866, 867, 868, 869, 870, 871, 872, 873, 874, 875, 876, 877, 878, 919, 920, 938, 939, 940, 941, 942, 943], 'keyabsage.wordpress.com': [7, 599, 600, 601, 618, 631], 'int.artloft.co': [8, 11, 12, 656, 657, 658, 659, 716, 717, 754, 982], 'dtlv.co': [9, 814], 'twitter.com': [14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 168, 169, 170, 171, 259, 260, 261, 463, 464, 465, 46

...

{'rferl.org': [0, 78, 105, 106, 150, 151, 152, 153, 154, 155], 'theglobaleye.it': [1, 44, 45], 'twitter.com': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 46], 'lrt.lt': [47], 'atlanticcouncil.org': [48, 81], 'carnegieendowment.org': [49], 'cepa.org': [50], 'recentlyheard.com': [51], 'themoscowtimes.com': [52, 53, 135, 136], 'bloomberg.com': [54], 'theguardian.com': [55, 58, 132], 'washingtonpost.com': [56], 'dw.com': [57, 73], 'meduza.io': [59], 'reuters.com': [60, 85, 134], 'dailymail.co.uk': [61], 'aljazeera.com': [62], 'en.news-front.info': [63], 'scmp.com': [64], 'liveuamap.com': [65, 76], 'medium.com': [66], 'veteranstoday.com': [67], 'southfront.org': [68], 'nbcnews.com': [69], 'cnbc.com': [70, 74], 'independent.co.uk': [71], 'breitbart.com': [72], 'thesun.co.uk': [75], 'apnews.com': [77, 84, 108], 'globalsecurity.org': [79, 109, 110], 'hri.org': [80, 87, 89, 90, 9

{'pl.sputniknews.com': [0, 1, 2, 3, 63, 64, 65, 68, 118, 125, 134, 137, 152, 165], 'fakt.pl': [4, 16, 51, 53, 60], 'tvp.info': [5, 41, 57, 90, 117], 'gazetaprawna.pl': [6, 18, 86, 87, 88], 'polityka.pl': [7, 49], '300polityka.pl': [8, 55], 'wiadomo.co': [9, 46, 59], 'wiadomosci.wp.pl': [10, 17, 40, 92], 'euractiv.pl': [11], 'wiadomosci.onet.pl': [12, 84, 139], 'oko.press': [13, 32, 62, 78], 'krytykapolityczna.pl': [14, 31, 158], 'wprost.pl': [15, 23, 43, 44], 'dorzeczy.pl': [19], 'gloria.tv': [20], 'wmeritum.pl': [21], 'trybuna.info': [22], 'bankier.pl': [24], 'gazetawarszawska.com': [25], 'wykop.pl': [26, 102], 'crowdmedia.pl': [27, 42, 47, 50, 52, 56, 103], 'gosc.pl': [28], 'kwejk.pl': [29], 'dzienniknarodowy.pl': [30], 'rp.pl': [33], 'kontrowersje.net': [34], 'forumdlazycia.wordpress.com': [35], 'pch24.pl': [36], 'propolski.pl': [37], 'polskieradio.pl': [38, 96], 'pl.news-front.info': [39], 'vnexplorer.net': [45, 111, 114, 124, 126, 133, 135], 'szostkiewicz.blog.polityka.pl': [48], 

{'ria.ru': [0, 138, 139, 156, 234, 302], 'armedconflicts.com': [1, 390, 416, 418], 'thebigtheone.com': [2, 236, 237, 238, 251, 252, 253], 'coolacniy.ru': [3, 7], 'tgstat.ru': [4, 279, 282], 'a2ch.ru': [5], 'guerresmondiales.com': [6, 363], 'valka.cz': [8, 391, 417, 430], 'twitter.com': [9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 169], 'kommersant.ru': [109], 'aif.ru': [110, 126, 165], 'lenta.ru': [111, 140, 287, 310], 'cryptofree.network': [112], '1prime.ru': [113], 'vedomosti.ru': [114, 313], 'ura.news': [115], 'fontanka.ru': [116, 168], 'mk.ru': [117], 'dumatv.ru': [118], 'iz.ru': [119], 'runews24.ru': [120], 'newsru.co.il': 

{'kp.ru': [0, 34, 37, 68], 'ria.ru': [1, 9, 50, 55], 'iz.ru': [2, 12, 35], 'gazeta.ru': [3, 5, 38, 61, 66, 71], 'lenta.ru': [4, 54, 63], 'm.vz.ru': [6], 'aif.ru': [7], 'rg.ru': [8, 51], 'interfax.ru': [10], 'inosmi.ru': [11], 'svpressa.ru': [13], 'argumenti.ru': [14], 'ren.tv': [15], 'osnmedia.ru': [16], 'ukraina.ru': [17], 'riafan.ru': [18, 33, 41, 44, 46], 'USA.one': [19], 'dumatv.ru': [20], 'mk.ru': [21], 'life.ru': [22, 28, 64], 'runews24.ru': [23, 62], 'tvzvezda.ru': [24], 'eadaily.com': [25, 49, 53, 56, 57, 60, 65], 'dailystorm.ru': [26], 'listaj.ru': [27], 'discover24.ru': [29], 'zakon.ru': [30], 'kommersant.ru': [31], 'sobesednik.ru': [32], 'vz.ru': [36, 39, 43, 48, 69], 'news.rin.ru': [40, 42, 58, 67, 72], 'yandex.ru': [45], '1prime.ru': [47], 'az.sputniknews.ru': [52], 'fontanka.ru': [59], 'bloknot.ru': [70]}
{'en.trend.az': [0, 52, 64], 'xinhuanet.com': [1], 'govinfo.gov': [2], 'en.royanews.tv': [3], 'tellerreport.com': [4, 42, 46, 47], 'zeenews.india.com': [5], 'citizen.co.

{'headtopics.com': [0], 'arabic.sputniknews.com': [1, 4], 'emirates.com': [2], 'blogger.com': [3], 'twitter.com': [5, 6, 7, 8]}
{'geopolitica.ru': [0, 1, 15, 16], 'twitter.com': [2, 3, 4], 'treccani.it': [5, 39], 'brainfactor.it': [6], 'ismu.org': [7], 'altalex.com': [8], 'academia.edu': [9, 14, 48, 49, 50, 57, 58, 60, 61], 'aatipalermo.uark.edu': [10], 'discountmags.com': [11], 'journals.openedition.org': [12], 'ibiblio.org': [13], 'pmi.it': [17], 'altuoservizio.conad.it': [18], 'ilfattoquotidiano.it': [19], 'salute.regione.veneto.it': [20], 'comedonchisciotte.org': [21], 'governo.it': [22], 'en.wikipedia.org': [23], 'italiana-russa.ru': [24], 'madrelingua.ru': [25], 'ilpost.it': [26], 'deepl.com': [27], 'netflix.com': [28], 'translator.eu': [29], 'vatican.va': [30], 'wumingfoundation.com': [31, 52], 'csoacartella.org': [32], 'commons.wikimedia.org': [33, 35], 'widenews.it': [34], 'pinterest.com': [36], 'interware.it': [37], 'international-communist-party.org': [38], 'disuguaglianzeso

In [4]:
train_graphs = [(dgl.from_networkx(g, node_attrs=["h"], edge_attrs=['type']), torch.tensor([l]).float()) for g, l in train_set]
train_labels = [l for _, l in train_set]
test_graphs = [(dgl.from_networkx(g, node_attrs=['h'], edge_attrs=['type']), torch.tensor([l]).float()) for g, l in test_set]
test_labels = [l for _, l in test_set]


In [5]:
class RGCN(nn.Module):
    def __init__(self, hidden_features, dropout, num_edge_types):
        super(RGCN, self).__init__()
        self.in_layer = RelGraphConv(3, hidden_features, num_edge_types, activation=nn.ReLU())
        self.dropout1 = nn.Dropout(dropout)
        self.out_layer = RelGraphConv(hidden_features, hidden_features, 3, activation=nn.ReLU())
        self.dropout2 = nn.Dropout(dropout)
        self.global_mean_pool = dgl.nn.pytorch.glob.AvgPooling()
        
        self.fc = nn.Linear(hidden_features, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, g, in_feat, e_types):
        h = self.in_layer(g, in_feat, e_types)
        h = self.dropout1(h)
        h = self.out_layer(g, h, e_types)
        h = self.dropout2(h)
        h = self.global_mean_pool(g, h)
        h = self.fc(h)
        
        return self.sigmoid(h)

In [6]:
def evaluate(model, dataloader):
    model.eval()
    num_correct = 0
    num_tests = 0
    for batched_graph, labels in dataloader:
        pred = torch.round(model(batched_graph, batched_graph.ndata['h'].float(), batched_graph.edata['type']))
        num_correct += (pred==labels).sum().float().item()
        num_tests += len(labels)
    return num_correct / num_tests

def train(model, train_loader, optimizer):
    Loss = nn.BCELoss()
    
    model.train()
    ret_loss = 0
    n_graphs = 0
    for data, target in train_loader:
        output = model(data, data.ndata['h'].float(), data.edata['type'])
        
        loss = Loss(output, target)
        ret_loss += loss.item()/len(target)
        n_graphs += len(target)     
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        return loss.item()
#     return ret_loss/n_graphs

def val_loss(model, loader):
    Loss = nn.BCELoss()
    model.eval()
    ret_loss = 0
    n_graphs = 0
    with torch.no_grad():
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            output = model(data, data.ndata['h'].float(), data.edata['type'])
            loss = Loss(output, target)

            return loss.item()
        


In [7]:
splits=StratifiedKFold(n_splits=5,shuffle=True,random_state=42)
split_idxs = splits.split(np.arange(len(train_graphs)), train_labels)

num_epochs=100
from itertools import product
hidden_size = [4, 8, 16, 32, 64]
dropout = [0., 0.3, 0.5, 0.7]
lr = [0.1, 0.05, 0.01, 0.005]

tg = dgl.batch([g for g,_ in train_graphs])
num_edge_types = tg.edata['type'].max().item() +1


model_params = list(product(hidden_size, dropout, lr))

# model_params = [[4, 0., 0.1]]
print(model_params)

[(4, 0.0, 0.1), (4, 0.0, 0.05), (4, 0.0, 0.01), (4, 0.0, 0.005), (4, 0.3, 0.1), (4, 0.3, 0.05), (4, 0.3, 0.01), (4, 0.3, 0.005), (4, 0.5, 0.1), (4, 0.5, 0.05), (4, 0.5, 0.01), (4, 0.5, 0.005), (4, 0.7, 0.1), (4, 0.7, 0.05), (4, 0.7, 0.01), (4, 0.7, 0.005), (8, 0.0, 0.1), (8, 0.0, 0.05), (8, 0.0, 0.01), (8, 0.0, 0.005), (8, 0.3, 0.1), (8, 0.3, 0.05), (8, 0.3, 0.01), (8, 0.3, 0.005), (8, 0.5, 0.1), (8, 0.5, 0.05), (8, 0.5, 0.01), (8, 0.5, 0.005), (8, 0.7, 0.1), (8, 0.7, 0.05), (8, 0.7, 0.01), (8, 0.7, 0.005), (16, 0.0, 0.1), (16, 0.0, 0.05), (16, 0.0, 0.01), (16, 0.0, 0.005), (16, 0.3, 0.1), (16, 0.3, 0.05), (16, 0.3, 0.01), (16, 0.3, 0.005), (16, 0.5, 0.1), (16, 0.5, 0.05), (16, 0.5, 0.01), (16, 0.5, 0.005), (16, 0.7, 0.1), (16, 0.7, 0.05), (16, 0.7, 0.01), (16, 0.7, 0.005), (32, 0.0, 0.1), (32, 0.0, 0.05), (32, 0.0, 0.01), (32, 0.0, 0.005), (32, 0.3, 0.1), (32, 0.3, 0.05), (32, 0.3, 0.01), (32, 0.3, 0.005), (32, 0.5, 0.1), (32, 0.5, 0.05), (32, 0.5, 0.01), (32, 0.5, 0.005), (32, 0.7, 0

In [8]:
# for l in splits.split(np.arange(len(train_graphs)), train_labels):
#     print(train_labels[l])

In [ ]:
best_test_acc = 0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
results = []
for i, (hidden_size, dropout, lr) in enumerate(model_params):
    foldperf = {}
    print(f"model {i}/{len(model_params)}")
    print('Model parameters: hidden_size {}, dropout {}, optimizer lr {}'.format(*model_params[i]))
    model = RGCN(hidden_size, dropout, num_edge_types)
    
    for fold, (train_idx, val_idx) in enumerate(splits.split(np.arange(len(train_graphs)), train_labels)):
        lrs = []


        train_sampler = SubsetRandomSampler(train_idx)
        test_sampler = SubsetRandomSampler(val_idx)
        train_loader = GraphDataLoader(train_graphs, batch_size=1000, sampler=train_sampler)
        test_loader = GraphDataLoader(train_graphs, batch_size=1000, sampler=test_sampler)

        optimizer = torch.optim.Adam(model.parameters(), lr=lr)
        history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}

        for epoch in range(num_epochs):
            train_loss = train(model,train_loader, optimizer)
            train_acc = evaluate(model, train_loader)
            test_loss = val_loss(model,test_loader)
            test_acc = evaluate(model, test_loader)
            if epoch%10==1:
                print("{} Loss train/val:{:.3f} / {:.3f} Acc train/val {:.2f} / {:.2f} %".format(epoch, train_loss,
                                                                                              test_loss,
                                                                                              100*train_acc,
                                                                                              100*test_acc))

            history['train_loss'].append(train_loss)
            history['test_loss'].append(test_loss)
            history['train_acc'].append(train_acc)
            history['test_acc'].append(test_acc)
        foldperf[f'fold{fold+1}'] = history  
    results.append(foldperf)
#     break


model 0/80
Model parameters: hidden_size 4, dropout 0.0, optimizer lr 0.1


C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:0.670 / 0.645 Acc train/val 68.59 / 70.00 %
11 Loss train/val:0.524 / 0.524 Acc train/val 71.79 / 67.50 %
21 Loss train/val:0.481 / 0.481 Acc train/val 73.08 / 70.00 %
31 Loss train/val:0.455 / 0.453 Acc train/val 76.28 / 67.50 %
41 Loss train/val:0.441 / 0.438 Acc train/val 77.56 / 75.00 %
51 Loss train/val:0.423 / 0.429 Acc train/val 80.13 / 72.50 %
61 Loss train/val:0.410 / 0.413 Acc train/val 78.85 / 75.00 %
71 Loss train/val:0.405 / 0.402 Acc train/val 79.49 / 70.00 %
81 Loss train/val:0.394 / 0.395 Acc train/val 78.21 / 72.50 %
91 Loss train/val:0.388 / 0.386 Acc train/val 79.49 / 72.50 %
1 Loss train/val:1.389 / 0.483 Acc train/val 78.98 / 74.36 %
11 Loss train/val:0.523 / 0.506 Acc train/val 75.80 / 79.49 %
21 Loss train/val:0.486 / 0.485 Acc train/val 75.16 / 74.36 %
31 Loss train/val:0.463 / 0.462 Acc train/val 77.07 / 79.49 %
41 Loss train/val:0.454 / 0.452 Acc train/val 77.71 / 74.36 %
51 Loss train/val:0.457 / 0.445 Acc train/val 77.07 / 69.23 %
61 Loss tr

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:3.021 / 2.542 Acc train/val 48.08 / 47.50 %
11 Loss train/val:0.696 / 0.689 Acc train/val 51.28 / 52.50 %
21 Loss train/val:0.667 / 0.665 Acc train/val 51.92 / 52.50 %
31 Loss train/val:0.663 / 0.662 Acc train/val 51.92 / 52.50 %
41 Loss train/val:0.659 / 0.658 Acc train/val 58.33 / 55.00 %
51 Loss train/val:0.656 / 0.656 Acc train/val 55.13 / 52.50 %
61 Loss train/val:0.652 / 0.652 Acc train/val 56.41 / 52.50 %
71 Loss train/val:0.648 / 0.647 Acc train/val 57.69 / 52.50 %
81 Loss train/val:0.641 / 0.640 Acc train/val 58.97 / 55.00 %
91 Loss train/val:0.634 / 0.634 Acc train/val 58.97 / 55.00 %
1 Loss train/val:0.641 / 0.643 Acc train/val 58.60 / 58.97 %
11 Loss train/val:0.632 / 0.637 Acc train/val 57.96 / 53.85 %
21 Loss train/val:0.633 / 0.630 Acc train/val 57.32 / 56.41 %
31 Loss train/val:0.651 / 0.647 Acc train/val 58.60 / 53.85 %
41 Loss train/val:0.636 / 0.636 Acc train/val 59.24 / 56.41 %
51 Loss train/val:0.640 / 0.639 Acc train/val 59.24 / 56.41 %
61 Loss tr

...

1 Loss train/val:7.920 / 7.016 Acc train/val 50.64 / 50.00 %
11 Loss train/val:8.324 / 8.062 Acc train/val 64.74 / 62.50 %
21 Loss train/val:5.021 / 4.063 Acc train/val 65.38 / 62.50 %
31 Loss train/val:7.192 / 1.248 Acc train/val 60.26 / 52.50 %
41 Loss train/val:1.236 / 9.984 Acc train/val 51.92 / 52.50 %
51 Loss train/val:5.195 / 5.374 Acc train/val 69.87 / 72.50 %
61 Loss train/val:1.649 / 1.653 Acc train/val 53.21 / 50.00 %
71 Loss train/val:2.979 / 2.422 Acc train/val 69.23 / 65.00 %
81 Loss train/val:5.852 / 6.041 Acc train/val 67.31 / 72.50 %
91 Loss train/val:5.916 / 5.768 Acc train/val 71.79 / 70.00 %
1 Loss train/val:17.010 / 13.005 Acc train/val 51.59 / 51.28 %
11 Loss train/val:2.447 / 1.627 Acc train/val 70.06 / 66.67 %
21 Loss train/val:0.939 / 4.548 Acc train/val 57.96 / 51.28 %
31 Loss train/val:4.290 / 4.217 Acc train/val 72.61 / 56.41 %
41 Loss train/val:2.015 / 1.414 Acc train/val 70.70 / 71.79 %
51 Loss train/val:4.562 / 4.447 Acc train/val 73.25 / 74.36 %
61 Loss 

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:5.528 / 5.283 Acc train/val 51.92 / 52.50 %
11 Loss train/val:5.078 / 1.386 Acc train/val 51.28 / 60.00 %
21 Loss train/val:0.690 / 0.887 Acc train/val 69.23 / 70.00 %
31 Loss train/val:0.784 / 0.672 Acc train/val 67.31 / 62.50 %
41 Loss train/val:0.637 / 0.636 Acc train/val 69.87 / 65.00 %
51 Loss train/val:0.632 / 0.608 Acc train/val 67.31 / 65.00 %
61 Loss train/val:0.603 / 0.595 Acc train/val 70.51 / 65.00 %
71 Loss train/val:0.589 / 0.583 Acc train/val 71.79 / 62.50 %
81 Loss train/val:0.575 / 0.574 Acc train/val 71.79 / 72.50 %
91 Loss train/val:0.566 / 0.565 Acc train/val 71.15 / 72.50 %
1 Loss train/val:1.421 / 1.028 Acc train/val 71.97 / 66.67 %
11 Loss train/val:0.686 / 0.626 Acc train/val 71.34 / 69.23 %
21 Loss train/val:0.566 / 0.600 Acc train/val 74.52 / 71.79 %
31 Loss train/val:0.569 / 0.558 Acc train/val 74.52 / 74.36 %
41 Loss train/val:0.556 / 0.551 Acc train/val 75.16 / 74.36 %
51 Loss train/val:0.546 / 0.547 Acc train/val 75.80 / 74.36 %
61 Loss tr

C:\Users\ondra\anaconda3\envs\cp\lib\site-packages\torch\autocast_mode.py:141: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


1 Loss train/val:14.056 / 15.937 Acc train/val 51.92 / 52.50 %
11 Loss train/val:31.521 / 30.508 Acc train/val 53.85 / 55.00 %
21 Loss train/val:16.040 / 16.003 Acc train/val 64.74 / 65.00 %
31 Loss train/val:12.130 / 11.655 Acc train/val 69.23 / 67.50 %
41 Loss train/val:0.973 / 0.952 Acc train/val 48.08 / 47.50 %
51 Loss train/val:0.735 / 0.720 Acc train/val 48.08 / 47.50 %
61 Loss train/val:0.705 / 0.707 Acc train/val 51.92 / 52.50 %
71 Loss train/val:0.707 / 0.704 Acc train/val 51.92 / 52.50 %
81 Loss train/val:0.694 / 0.694 Acc train/val 50.64 / 50.00 %
91 Loss train/val:0.695 / 0.695 Acc train/val 48.08 / 47.50 %
1 Loss train/val:0.694 / 0.692 Acc train/val 51.59 / 53.85 %
11 Loss train/val:0.681 / 0.686 Acc train/val 51.59 / 53.85 %
21 Loss train/val:0.688 / 0.685 Acc train/val 51.59 / 53.85 %


In [ ]:
for i, foldperf in enumerate(results):
    train_loss = np.vstack([foldperf[fold]['train_loss'] for fold in foldperf])
    test_loss = np.vstack([foldperf[fold]['test_loss'] for fold in foldperf])
    train_acc = np.vstack([foldperf[fold]['train_acc'] for fold in foldperf])
    test_acc = np.vstack([foldperf[fold]['test_acc'] for fold in foldperf])
    
    fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(9, 9))
    fig.suptitle(f'Model {i}, hidden_size {model_params[i][0]}, dropout {model_params[i][1]}, lr {model_params[i][2]}')
    ax1.set_xlabel("Epoch")
    ax1.set_ylabel("Loss")
    ax1.title.set_text('Training loss')
    ax1.fill_between(range(num_epochs), train_loss.min(0), train_loss.max(0), alpha=0.3, label="train loss <min; max>")
    ax1.fill_between(range(num_epochs), test_loss.min(0), test_loss.max(0), alpha=0.3, label="test loss <min; max>")

    ax1.plot(range(num_epochs), train_loss.mean(0), label='train loss mean')
    ax1.plot(range(num_epochs), test_loss.mean(0), label='test loss mean')
    ax1.legend()
    
    
    
    ax2.title.set_text("Training loss with cross validation")
    ax2.set_xlabel("Epoch")
    ax2.set_ylabel("Accuracy")
    ax2.fill_between(range(num_epochs), train_acc.min(0), train_acc.max(0), alpha=0.2, label="train acc <min; max>")
    ax2.fill_between(range(num_epochs), test_acc.min(0), test_acc.max(0), alpha=0.2, label="test acc <min; max>")

    ax2.plot(range(num_epochs), train_acc.mean(0))

    ax2.plot(range(num_epochs), test_acc.mean(0))
    ax2.legend()

    
    
    ax3.title.set_text('Train loss per fold')
    for l in train_loss:
        ax3.plot(l)
#     for l in train_loss:
#         ax3.plot(l, 'b-', alpha=0.5)
#     for l in test_loss:
#         ax3.plot(l, 'r-', alpha=0.5)
#     ax3.plot(np.convolve(train_loss.mean(0), np.ones(20)/20, mode='valid'), label='train loss')
#     ax3.plot(np.convolve(test_loss.mean(0), np.ones(20)/20, mode='valid'), label='test loss')
    ax3.set_xlabel('epoch')
    ax3.set_ylabel('running average loss')

    ax3.legend()
    
    ax4.title.set_text('Validation loss per fold')
    for l in test_loss:
        ax4.plot(l)
#     ax3.plot(np.convolve(train_loss.mean(0), np.ones(20)/20, mode='valid'), label='train loss')
#     ax3.plot(np.convolve(test_loss.mean(0), np.ones(20)/20, mode='valid'), label='test loss')
    ax4.set_xlabel('epoch')
    ax4.set_ylabel('Accuracy')
    plt.show()


In [ ]:
accs = []
for i, foldperf in enumerate(results):
    accs.append(np.vstack([foldperf[fold]['test_acc'][-1] for fold in foldperf]).mean(0))
best_params = model_params[np.asarray(accs).argmax()]



In [ ]:
hidden_size, dropout, lr = best_params
model = RGCN(hidden_size, dropout, num_edge_types)

train_loader = GraphDataLoader(train_graphs, batch_size=1000)
# test_loader = GraphDataLoader(train_graphs, batch_size=1000)

optimizer = torch.optim.Adam(model.parameters(), lr=lr)
history = {'train_loss': [], 'test_loss': [],'train_acc':[],'test_acc':[]}
train_loss = []
train_acc = []

for epoch in range(num_epochs):
    loss = train(model,train_loader,optimizer)
    acc = evaluate(model, train_loader)



    train_loss.append(loss)
    train_acc.append(acc)


In [ ]:
plt.figure()
fig, (ax1, ax2) = plt.subplots(1,2,figsize=(10, 5))
plt.suptitle('Training loss and acc for the best model')
ax1.set_xlabel('eppch')
ax1.set_ylabel('loss')
ax1.plot(train_loss)

ax2.set_xlabel('epoch')
ax2.set_ylabel('Acc [%]')
ax2.plot(train_acc)

plt.show()

In [ ]:
###### test_loader = GraphDataLoader(test_graphs, batch_size=1000)

print('best params: hidden size {}, dropout {}, Adam learning rate{}'.format(*best_params))
acc = evaluate(model, test_loader)
print(f'Model test acc is {acc*100}%')